In [53]:
from openai import OpenAI
import os
import stockhelper
import json
from dotenv import load_dotenv
import time

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_KEY"))

In [54]:
assistant_id = 'asst_4xBZF4HGIDBiUeN46uWsNa6B'

In [55]:
assistant = client.beta.assistants.retrieve(assistant_id)

assistant

Assistant(id='asst_4xBZF4HGIDBiUeN46uWsNa6B', created_at=1710332508, description=None, file_ids=[], instructions='Get the current stock price of a given symbol. If function returns None then the symbol is invalid.', metadata={}, model='gpt-3.5-turbo', name='Stock Price Assistant', object='assistant', tools=[ToolFunction(function=FunctionDefinition(name='get_stock_price', description='Get the current price of a stock for a given symbol. If function returns None then the symbol is invalid.', parameters={'type': 'object', 'properties': {'symbol': {'type': 'string', 'description': 'The stock symbol'}}, 'required': ['symbol']}), type='function')])

In [56]:
thread = client.beta.threads.create()
thread

Thread(id='thread_VD4vFCfn5IAyT7uby6Tda5vs', created_at=1710405488, metadata={}, object='thread')

In [57]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant_id,
    instructions = "What is the price of Nvidia stock?"
)
run
time.sleep(3)

In [58]:
run = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id=run.id
)
run

Run(id='run_9lqzFdu5X0TlX5cOhGmUUKo2', assistant_id='asst_4xBZF4HGIDBiUeN46uWsNa6B', cancelled_at=None, completed_at=None, created_at=1710405489, expires_at=1710406089, failed_at=None, file_ids=[], instructions='What is the price of Nvidia stock?', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_uVLF6c7wGKncEemGzB4VzXpe', function=Function(arguments='{"symbol":"NVDA"}', name='get_stock_price'), type='function')]), type='submit_tool_outputs'), started_at=1710405489, status='requires_action', thread_id='thread_VD4vFCfn5IAyT7uby6Tda5vs', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='get_stock_price', description='Get the current price of a stock for a given symbol. If function returns None then the symbol is invalid.', parameters={'type': 'object', 'properties': {'symbol': {'type': 'string', 'description': 'T

In [59]:
def get_output_for_tool_call(tool_call):
    stock_symbol = json.loads(tool_call.function.arguments)["symbol"]
    stock_details = stockhelper.get_stock_price(stock_symbol)
    return {
        "tool_call_id": tool_call.id,
        "output": stock_details
    }
    
tool_calls = run.required_action.submit_tool_outputs.tool_calls
tool_outputs = map(get_output_for_tool_call, tool_calls)
tool_outputs = list(tool_outputs)
tool_outputs

[{'tool_call_id': 'call_uVLF6c7wGKncEemGzB4VzXpe',
  'output': '{"symbol": "NVDA", "current_price": 908.88, "previous_close_price": 919.13}'}]

In [60]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id = thread.id,
    run_id = run.id,
    tool_outputs = tool_outputs
)
run
time.sleep(1)

In [61]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)
messages.data[0].content[0].text.value

'The current price of Nvidia (NVDA) stock is $908.88.'